<a href="https://colab.research.google.com/github/ssawant/TensorFlow-in-Practice/blob/main/C3W4_NLP_Reading_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.4.0


In [3]:
file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(file, 'rb').read().decode(encoding='utf-8')

print(f"{len(text)}")
print(f"{text[:250]}")

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [44]:
vocab = sorted(set(text))
print(f"{len(vocab)}")
# print(f"{vocab}")

65


In [32]:
# creating a mapping for unique characters to indices
char2indx = {u:i for i, u in enumerate(vocab)}
indx2char = np.array(vocab)

text_as_int = np.array([char2indx[c] for c in text])
# print(f"{len(text_as_int)}")
print(f"{char2indx['C']}")
print(f"{text[:13]} ===> {text_as_int[:13]}")

15
First Citizen ===> [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [39]:
seq_length = 100
example_per_epoch = len(text)

# Creating training example/ traget
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(15):
  # print(i.numpy())
  print(indx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i
z
e
n
:




In [40]:
sequence = char_dataset.batch(seq_length+1, drop_remainder=True)

print(sequence.take(13))

<TakeDataset shapes: (101,), types: tf.int64>


In [42]:
# return two list with first and last char removed
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequence.map(split_input_target) # lambda function call

for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(indx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(indx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [43]:
# Creating training batch
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [48]:
vocab_size = len(vocab) # 65
embedding_dim = 256
rnn_units = 1024

model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[BATCH_SIZE, None]),
  tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
  tf.keras.layers.Dense(vocab_size)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
